In [37]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key

#### Import key API code idea from Nicole Lund. Shared in UofA-PHX-VIRT-DATA-PT-03-2021, Slack: Resources Channel

In [38]:

# Import API key
from api_keys import g_key
if g_key == "YOUR KEY HERE!":
    import sys
    sys.path.append(r"/Users/rosaicelaroman/Desktop/Data_BootCamp/APIs/config_keys/")
    from api_keys_v2 import g_key

#### Read csv and create dataframe containing all the information collected in the weather notebook.

In [39]:
all_weather_df = pd.read_csv('all_weather.csv')
all_weather_df.head(10)

,cities,lat,lng,temp,humidity,cloudiness,wind
0,souillac,-20.5167,57.5167,25.00,88,75,4.63
1,balikpapan,-1.2675,116.8289,29.32,63,74,3.53
2,saint anthony,45.0205,-93.2180,7.18,81,90,2.06
3,atuona,-9.8000,-139.0333,26.84,77,22,10.68
4,nome,64.5011,-165.4064,3.15,70,1,1.20
5,cidreira,-30.1811,-50.2056,20.66,83,1,6.77
6,punta arenas,-53.1500,-70.9167,5.78,93,75,1.03
7,santa isabel do rio negro,-0.4139,-65.0192,23.21,97,85,0.88
8,rikitea,-23.1203,-134.9692,26.08,75,100,6.66
9,tuatapere,-46.1333,167.6833,12.22,75,49,2.68


#### Extracting the latitude and longitude from dataframe to be used as locations in gmaps. Extracting humidity to be used as weights in gmaps.

In [40]:
locations = all_weather_df[['lat', 'lng']]
# max_hum = all_weather_df['humidity'].max()
# print(max_hum)
humidity = all_weather_df['humidity'].astype(float)

#### Create gmap: figure was centered at latitude 18 and longitude 0 to provide a more centralized view of the map. zoom level was set to 2 to remove some of the extra view in the image. Max intensity was set to 100 to match the max possible relative humidity. Radius was increased to 3.

In [41]:
 # Plot Heatmap

gmaps.configure(api_key = g_key)
fig = gmaps.figure()

fig = gmaps.figure(layout={
        'width': '400px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'
})
# Create heat layer
fig = gmaps.figure(center=(18.0, 0.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)



# Add layer
fig.add_layer(heat_layer)

# Display figure
fig        

Figure(layout=FigureLayout(height='420px'))

#### Below, original weather dataframe is bein modified for the ideal weather filtering:
    1. Changing datatypes to floats for the numeric values.
    2. Removing unnecessary columns.
    

In [42]:
temp_float = all_weather_df['temp'].astype(float)
wind_float = all_weather_df['wind'].astype(float)
cloud_float = all_weather_df['cloudiness'].astype(float)
all_weather_df = all_weather_df.drop(['temp'], axis=1)
all_weather_df = all_weather_df.drop(['wind'], axis=1)
all_weather_df = all_weather_df.drop(['cloudiness'], axis=1)
all_weather_df.head()

,cities,lat,lng,humidity
0,souillac,-20.5167,57.5167,88
1,balikpapan,-1.2675,116.8289,63
2,saint anthony,45.0205,-93.2180,81
3,atuona,-9.8000,-139.0333,77
4,nome,64.5011,-165.4064,70


In [43]:
conv_all_weather = all_weather_df.copy()
conv_all_weather['temp'] = temp_float
conv_all_weather['wind'] = wind_float
conv_all_weather['cloudiness'] = cloud_float
conv_all_weather.head()

,cities,lat,lng,humidity,temp,wind,cloudiness
0,souillac,-20.5167,57.5167,88,25.00,4.63,75.0
1,balikpapan,-1.2675,116.8289,63,29.32,3.53,74.0
2,saint anthony,45.0205,-93.2180,81,7.18,2.06,90.0
3,atuona,-9.8000,-139.0333,77,26.84,10.68,22.0
4,nome,64.5011,-165.4064,70,3.15,1.20,1.0


#### Below, the data was actually filtered for places with temperature between 68ºF and ~85ºF (converted to Celsius to match the dataframe), cloudiness less than 10% and wind less than 5 mph.

In [44]:
ideal_weather = conv_all_weather[(conv_all_weather['temp'] > 20) & (conv_all_weather['temp'] < 29.44) & (conv_all_weather['cloudiness'] < 10) & (conv_all_weather['wind'] < 5)]
ideal_weather.head()

,cities,lat,lng,humidity,temp,wind,cloudiness
53,ixtapa,20.7000,-105.2000,78,24.22,1.17,1.0
62,kaina,12.6374,8.5873,47,28.00,4.12,0.0
72,saint-philippe,-21.3585,55.7679,73,22.81,3.60,0.0
76,sirsa,29.5333,75.0167,28,27.31,3.56,0.0
109,carnarvon,-24.8667,113.6333,42,29.00,3.09,2.0


#### Below the new dataframe for ideal weather had columns added for the additional information to be retrieved from google places.

In [45]:
ideal_weather['Hotel Name'] = ''
ideal_weather['Hotel Lat'] = ""
ideal_weather['Hotel Long'] = ''
ideal_weather['Hotel Address'] = ''
ideal_weather['Hotel City'] = ''
ideal_weather['Hotel Country'] = ''
ideal_weather.head()

,cities,lat,lng,humidity,temp,wind,cloudiness,Hotel Name,Hotel Lat,Hotel Long,Hotel Address,Hotel City,Hotel Country
53,ixtapa,20.7000,-105.2000,78,24.22,1.17,1.0,,,,,,
62,kaina,12.6374,8.5873,47,28.00,4.12,0.0,,,,,,
72,saint-philippe,-21.3585,55.7679,73,22.81,3.60,0.0,,,,,,
76,sirsa,29.5333,75.0167,28,27.31,3.56,0.0,,,,,,
109,carnarvon,-24.8667,113.6333,42,29.00,3.09,2.0,,,,,,


#### Loop to test the response and explore the information in json

In [46]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in ideal_weather.iterrows():
    key = g_key
    location = str(row['lat'])+ ","+str(row['lng'])
    radius = 5000
    types = 'lodging'
    params = {
        'location': location,
        'radius': radius,
        'type': types,
        'key': key
    }
    print(f'Retrieving results for {index}') 
    response = requests.get(base_url, params=params).json()

    results = response['results']
    print(json.dumps(response, indent=4, sort_keys=True))
    break

Retrieving results for 53
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 20.7150301,
                    "lng": -105.2056156
                },
                "viewport": {
                    "northeast": {
                        "lat": 20.7163865802915,
                        "lng": -105.2042653697085
                    },
                    "southwest": {
                        "lat": 20.7136886197085,
                        "lng": -105.2069633302915
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Hotel Plaza Providencia",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 1908,
                    "html_attributions

#### Loop to populate the Hotel Name, Latitude, Longitude and Address using places api nearby search.

In [47]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in ideal_weather.iterrows():
    key = g_key
    location = str(row['lat'])+ ","+str(row['lng'])
    radius = 5000
    types = 'lodging'
    params = {
        'location': location,
        'radius': radius,
        'type': types,
        'key': key
    }
    print(f'Retrieving results for {index}') 
    response = requests.get(base_url, params=params).json()

    results = response['results']

    try:
        # print(f'Closest {types} is {results[0]['name']}.')
        ideal_weather.loc[index, 'Hotel Name']= results[0]['name']
        ideal_weather.loc[index, 'Hotel Lat']= results[0]['geometry']['location']['lat']
        ideal_weather.loc[index, 'Hotel Long']= results[0]['geometry']['location']['lng']
        ideal_weather.loc[index, 'Hotel Address']= results[0]['vicinity']

    except (KeyError, IndexError):
        print('Missing field/result... skipping.')
        ideal_weather.loc[index, 'Hotel Name']= float('NaN')
        ideal_weather.loc[index, 'Hotel Lat']= float('NaN')
        ideal_weather.loc[index, 'Hotel Long']= float('NaN')
        ideal_weather.loc[index, 'Hotel Address']= float('NaN')
    print("---------------")


Retrieving results for 53
---------------
Retrieving results for 62
Missing field/result... skipping.
---------------
Retrieving results for 72
---------------
Retrieving results for 76
---------------
Retrieving results for 109
---------------
Retrieving results for 119
---------------
Retrieving results for 120
---------------
Retrieving results for 121
---------------
Retrieving results for 170
---------------
Retrieving results for 195
---------------
Retrieving results for 219
---------------
Retrieving results for 238
---------------
Retrieving results for 243
Missing field/result... skipping.
---------------
Retrieving results for 278
Missing field/result... skipping.
---------------
Retrieving results for 289
---------------
Retrieving results for 298
---------------
Retrieving results for 301
---------------
Retrieving results for 310
---------------
Retrieving results for 331
---------------
Retrieving results for 343
---------------
Retrieving results for 410
---------------

In [48]:
ideal_weather.head()

,cities,lat,lng,humidity,temp,wind,cloudiness,Hotel Name,Hotel Lat,Hotel Long,Hotel Address,Hotel City,Hotel Country
53,ixtapa,20.7000,-105.2000,78,24.22,1.17,1.0,Hotel Plaza Providencia,20.715,-105.206,"Avenida Independencia 188, Centro Ixtapa, Ixtapa",,
62,kaina,12.6374,8.5873,47,28.00,4.12,0.0,NaN,NaN,NaN,NaN,,
72,saint-philippe,-21.3585,55.7679,73,22.81,3.60,0.0,"Chambres d'hôte ""La Trinité""",-21.3584,55.7703,12 Rue De La Pompe,,
76,sirsa,29.5333,75.0167,28,27.31,3.56,0.0,OYO 48542 Hotel Capital,29.5324,75.0373,"110, 10km stone, Hisar Road, Sirsa",,
109,carnarvon,-24.8667,113.6333,42,29.00,3.09,2.0,Hospitality Carnarvon,-24.8923,113.655,"6 West Street, South Carnarvon",,


#### Below is the loop using hotel latitude and longitude to do a reverse geocode search to find hotel city and hotel country.

In [49]:
base_url = "https://maps.googleapis.com/maps/api/geocode/json"
for index, row in ideal_weather.iterrows():
    key = g_key
    coord = str(row['Hotel Lat'])+ ','+str(row['Hotel Long'])
    # radius = 5000
    # types = 'lodging'
    params = {
        'latlng': coord,
        'locality': 'political',
        'location_type': "APPROXIMATE",
        'key': key
    }
    print(f'Retrieving results for {index}') 
    response = requests.get(base_url, params=params).json()

    results = response['results']
    
    # print(json.dumps(results[0]['address_components'][3]['short_name'], indent=4, sort_keys=True))
    # break
# 
    # # print(json.dumps((results[0]['formatted_address']), indent=4, sort_keys=True))
    # break
    try:
        # print(f'Closest {types} is {results[0]['name']}.')
        
        ideal_weather.loc[index, 'Hotel City']= results[0]['address_components'][1]['long_name']
        ideal_weather.loc[index, 'Hotel Country']= results[0]['address_components'][3]['long_name']
        
    except (KeyError, IndexError):
        print('Missing field/result... skipping.')
        ideal_weather.loc[index, 'Hotel City']= float('NaN')
        ideal_weather.loc[index, 'Hotel Country']= float('NaN')
    print("---------------")
   


Retrieving results for 53
---------------
Retrieving results for 62
Missing field/result... skipping.
---------------
Retrieving results for 72
Missing field/result... skipping.
---------------
Retrieving results for 76
---------------
Retrieving results for 109
---------------
Retrieving results for 119
---------------
Retrieving results for 120
---------------
Retrieving results for 121
---------------
Retrieving results for 170
---------------
Retrieving results for 195
---------------
Retrieving results for 219
---------------
Retrieving results for 238
---------------
Retrieving results for 243
Missing field/result... skipping.
---------------
Retrieving results for 278
Missing field/result... skipping.
---------------
Retrieving results for 289
---------------
Retrieving results for 298
---------------
Retrieving results for 301
Missing field/result... skipping.
---------------
Retrieving results for 310
---------------
Retrieving results for 331
---------------
Retrieving result

In [50]:
ideal_weather

,cities,lat,lng,humidity,temp,wind,cloudiness,Hotel Name,Hotel Lat,Hotel Long,Hotel Address,Hotel City,Hotel Country
53,ixtapa,20.7000,-105.2000,78,24.22,1.17,1.0,Hotel Plaza Providencia,20.715,-105.206,"Avenida Independencia 188, Centro Ixtapa, Ixtapa",Ixtapa,Mexico
62,kaina,12.6374,8.5873,47,28.00,4.12,0.0,NaN,NaN,NaN,NaN,NaN,NaN
72,saint-philippe,-21.3585,55.7679,73,22.81,3.60,0.0,"Chambres d'hôte ""La Trinité""",-21.3584,55.7703,12 Rue De La Pompe,NaN,NaN
76,sirsa,29.5333,75.0167,28,27.31,3.56,0.0,OYO 48542 Hotel Capital,29.5324,75.0373,"110, 10km stone, Hisar Road, Sirsa",Sirsa,Haryana
109,carnarvon,-24.8667,113.6333,42,29.00,3.09,2.0,Hospitality Carnarvon,-24.8923,113.655,"6 West Street, South Carnarvon",Shire of Carnarvon,Australia
119,kefamenanu,-9.4467,124.4781,58,28.85,2.54,6.0,Sokowindo Losmen,-9.4468,124.478,"JL Kartini, No No.3, Kefamenanu Tengah",Kefamenanu Tengah,North Timor Tengah Regency
120,bandiagara,14.3501,-3.6104,50,27.00,2.06,0.0,Hotel de la Falaise,14.3518,-3.61542,Mali,Bandiagara,Mali
121,rio branco,-9.9747,-67.8100,100,23.00,1.03,0.0,João Paulo Hotel,-9.97045,-67.8153,"Avenida Ceará, 2090 - Base, Rio Branco",Avenida Ceará,Rio Branco
170,caravelas,-17.7125,-39.2481,88,22.72,1.48,6.0,Pousada dos Navegantes,-17.738,-39.2589,"Rua das Palmeiras, 45, Caravelas",Caravelas,Brazil
195,atar,20.5169,-13.0499,17,23.93,2.91,0.0,Oumou Elghoura,20.5224,-13.0504,"Mbarka Oumara Derriere l'hopital regional, Atar",Atar,Mauritania


#### Below, the dataframe was cleaned of NaN values.

In [51]:
ideal_weather.dropna(inplace=True)
ideal_weather

,cities,lat,lng,humidity,temp,wind,cloudiness,Hotel Name,Hotel Lat,Hotel Long,Hotel Address,Hotel City,Hotel Country
53,ixtapa,20.7000,-105.2000,78,24.22,1.17,1.0,Hotel Plaza Providencia,20.715,-105.206,"Avenida Independencia 188, Centro Ixtapa, Ixtapa",Ixtapa,Mexico
76,sirsa,29.5333,75.0167,28,27.31,3.56,0.0,OYO 48542 Hotel Capital,29.5324,75.0373,"110, 10km stone, Hisar Road, Sirsa",Sirsa,Haryana
109,carnarvon,-24.8667,113.6333,42,29.00,3.09,2.0,Hospitality Carnarvon,-24.8923,113.655,"6 West Street, South Carnarvon",Shire of Carnarvon,Australia
119,kefamenanu,-9.4467,124.4781,58,28.85,2.54,6.0,Sokowindo Losmen,-9.4468,124.478,"JL Kartini, No No.3, Kefamenanu Tengah",Kefamenanu Tengah,North Timor Tengah Regency
120,bandiagara,14.3501,-3.6104,50,27.00,2.06,0.0,Hotel de la Falaise,14.3518,-3.61542,Mali,Bandiagara,Mali
121,rio branco,-9.9747,-67.8100,100,23.00,1.03,0.0,João Paulo Hotel,-9.97045,-67.8153,"Avenida Ceará, 2090 - Base, Rio Branco",Avenida Ceará,Rio Branco
170,caravelas,-17.7125,-39.2481,88,22.72,1.48,6.0,Pousada dos Navegantes,-17.738,-39.2589,"Rua das Palmeiras, 45, Caravelas",Caravelas,Brazil
195,atar,20.5169,-13.0499,17,23.93,2.91,0.0,Oumou Elghoura,20.5224,-13.0504,"Mbarka Oumara Derriere l'hopital regional, Atar",Atar,Mauritania
219,riyadh,24.6877,46.7219,15,26.00,2.57,0.0,"InterContinental Riyadh, an IHG Hotel",24.6667,46.6922,"King Saud Road, Riyadh",Riyadh,Riyadh Province
238,coffs harbour,-30.3000,153.1333,49,23.00,4.63,0.0,Aquajet Motel,-30.2849,153.131,"49 Park Beach Road, Coffs Harbour",Coffs Harbour City Council,Australia


#### Create list for the hotel name, city, country from dataframe columns.

In [52]:
hotel_name = ideal_weather['Hotel Name'].tolist()
hotel_city = ideal_weather['Hotel City'].tolist()
hotel_country = ideal_weather['Hotel Country'].tolist()

#### Below, the heatmap made in the previous lines will be modified to contain markers for the hotel found:
    1. A list of markers locations was created from the hotel lat longitude.
    2. gmaps was configured to call the key and figure was created.
    3. List was initiated to contain the strings needed for info_box_content.
    4. Initiated variable and created while loop for the length of marker locations and create a string containing the info for the content box.
    5. Added markers layers to the gmap.

In [59]:
 # Plot Heatmap
marker_locations = ideal_weather[['Hotel Lat', 'Hotel Long']]
gmaps.configure(api_key = g_key)
fig = gmaps.figure()

box_info = []
i=0
while i != len(marker_locations):
    box_info.append(f'HOTEL: {hotel_name[i]}, CITY: {hotel_city[i]}, COUNTRY: {hotel_country[i]}')
    i+=1

# print(box_info)
# print(len(box_info))
# print(len(marker_locations))

fig = gmaps.figure(layout={
        'width': '400px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'
})
# Create heat layer
fig = gmaps.figure(center=(18.0, 0.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)
markers = gmaps.marker_layer(marker_locations, info_box_content=box_info)

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(markers)
fig

     

Figure(layout=FigureLayout(height='420px'))